In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Specify the path to your CSV file
csv_file = 'results/scores_test2.csv'
df = pd.read_csv(csv_file)
df = df.drop(["run_name", "max_iters"], axis=1)

In [ ]:
df.head()

In [ ]:
# Define the grouping keys
group_keys = ["dataset", "model_type", "num_layers", "readout_type"]

# Define the metrics for which statistics should be computed
metrics = ["acc", "acc_foreground", "iou", "iou_foreground", "loss", "loss_foreground"]

# Group by the specified keys and compute statistics across the "extension" category
grouped_df = df.groupby(group_keys).agg({
    metric: ['mean', 'std'] for metric in metrics
}).reset_index()

In [ ]:
grouped_df

In [ ]:
grouped_df.to_csv("results/scores_processed1.csv", index=True)

In [ ]:
pivoted_df = grouped_df.pivot_table(index=group_keys)#, columns='extension', values=metrics, aggfunc='mean')

In [ ]:
pivoted_df

In [ ]:
pivoted_df = pivoted_df.round(2)

In [ ]:
pivoted_df

In [ ]:
datasets = ["mnist", "new_tetronimoes"]
models = ["cornn_model2", "conv_recurrent2", "baseline1_flexible"]
num_layers = [2, 4, 8, 16, 32]
readouts = ["linear", "fft", "last", "mean_time", "max_time"]

In [ ]:
results = {}
for dataset in datasets:
    if dataset not in results:
        results[dataset] = {}
    for model in models:
        if model not in results[dataset]:
            results[dataset][model] = {}
        if model == "baseline1_flexible":
            for layer in num_layers:
                df1 = pivoted_df.loc[(dataset, model, layer, "linear")]
                results[dataset][model][layer] = df1
        else:
            for readout in readouts:
                df1 = pivoted_df.loc[(dataset, model, 16, readout)]
                results[dataset][model][readout] = df1

In [ ]:
def do_concats(dataset, models):
    df1 = pd.concat(results[dataset][models[2]], axis=1).T
    df2 = pd.concat(results[dataset][models[1]], axis=1).T
    df3 = pd.concat(results[dataset][models[0]], axis=1).T
    return pd.concat((df1, df2, df3), axis=0, keys=("CNN", "LSTM", "CoRNN"))

In [ ]:
df1 = do_concats("mnist", models)
df2 = do_concats("new_tetronimoes", models)

In [ ]:
df1 = pd.concat((df1, df2), axis=0, keys=("MNIST", "Tetrominoes"))

In [ ]:
df1

In [ ]:
df1.to_csv("results/scores_processed3.csv", index=True)
df1.to_latex("results/scores_processed3", index=True, float_format="%.2f")